In [3]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

Taken a random url from wikipedia that has the necessary data

In [4]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"

In [5]:
data=requests.get(url).text
soup=BeautifulSoup(data,"html5lib")
tables=soup.find('table')

Create a dataframe to contain the required data. 

In [6]:
neighborhood_data = pd.DataFrame(columns=["PostalCode","Borough","Neighborhood"])

lastpostalcode=""
lastborough=""
lastneighborhood=""
for row in soup.find("tbody").find_all("tr"):
    
    col = row.find_all("td")
    
    postalcode=""
    borough=""
    neighborhood=""
    
    #If col is not populated then no need to populate the values in the data frame - so loop back
    if (col == []):
        continue
        
    #Assign the current col values to appropriate variables
    postalcode = col[0].text
    borough = col[1].text.strip()
    neighborhood = col[2].text.strip()

    #If borough is not assigned then no need to populate the values in the data frame - so loop back
    if borough == "Not assigned":
        continue
        
    #If neighborhood is not assigned then populate neighborhood as the borough
    if neighborhood == "Not assigned":
        neighborhood=borough

    #If postal code is same as the last row postal code then combine both neighborhood and continue
    if postalcode == lastpostalcode:
        lastneighborhood = lastneighborhood + ", " + neighborhood
        continue
    
    #No need to write when the first row is being read
    if lastpostalcode=="":
        pass
    #If control reaches here then all values OK and new postal code started - so to be written to the dataframe   
    else:
        neighborhood_data = neighborhood_data.append({"PostalCode":lastpostalcode, "Borough":lastborough, "Neighborhood":lastneighborhood}, ignore_index=True)
        
    #Assign the new values to the last variables
    lastpostalcode=postalcode
    lastborough=borough
    lastneighborhood=neighborhood
pd.set_option('display.max_colwidth',None)    
neighborhood_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
97,M5X,Downtown Toronto,"First Canadian Place, Underground city"
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern


In [8]:
neighborhood_data.shape

(102, 3)

In [ ]:
!pip install wheel
!pip install pipwin

!pipwin install numpy
!pipwin install pandas
!pipwin install shapely
!pipwin install gdal
!pipwin install fiona
!pipwin install pyproj
!pipwin install six
!pipwin install rtree
!pipwin install geopandas
#!pip uninstall geopy
import geocoder # import geocoder

In [ ]:
# initialize your variable to None
lat_lng_coords = None
print("hello222")
# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    print("hello11")

print("hello")
latitude = lat_lng_coords[0]
print(latitude)
longitude = lat_lng_coords[1]

Import geocode.csv in to dataframe

In [9]:
df_geocode=pd.read_csv('Geospatial_Coordinates.csv')
df_geocode.head(15)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Rename the Postal Code column in geocode dataframe to match that of neighborhood dataframe

In [28]:
df_geocode.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [33]:
df_geocode.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Merge the 2 dataframes to get the desired result

In [44]:
df_nbhd_toronto = neighborhood_data.merge(df_geocode, how='left')

In [45]:
df_nbhd_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Install all required packages

In [38]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\snata\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             con

Get geographical coordinate of Toronto city

In [41]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto city are 43.6534817, -79.3839347.


Create map of Toronto using latitude and longitude values

In [46]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_nbhd_toronto['Latitude'], df_nbhd_toronto['Longitude'], df_nbhd_toronto['Borough'], df_nbhd_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

A. We will segment and cluster only the neighborhoods of Etobicoke

Create a new dataframe from the df_nbhd_toronto dataframe filtering data for borough Etobicoke
Get the geographical coordinates of Etobicoke
Create a map for the same
Utilize Foursquare API to explore the neighborhoods and segment those


In [52]:
etobicoke_data = df_nbhd_toronto[df_nbhd_toronto['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",43.650943,-79.554724
2,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",43.688905,-79.554724
5,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
6,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",43.739416,-79.588437
7,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
8,M9W,Etobicoke,Northwest,43.706748,-79.594054
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


Let's get the geographical co-ordinates of Etobicoke

In [53]:
address = 'Etobicoke'

geolocator = Nominatim(user_agent="etobicoke_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


Visualize the neighborhood of Etobicoke

In [55]:
# create map of Manhattan using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

In [57]:
CLIENT_ID = 'FKGQWKIH4UQDZB0CMZFIHK5AQ42PN4TE05OAFCFEFUXJVF44' # your Foursquare ID
CLIENT_SECRET = 'K3CU2TJC0XDSTXDKCNEKZ1ZP2GJFYNNMPSQOWVNQ5U4COEFN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FKGQWKIH4UQDZB0CMZFIHK5AQ42PN4TE05OAFCFEFUXJVF44
CLIENT_SECRET:K3CU2TJC0XDSTXDKCNEKZ1ZP2GJFYNNMPSQOWVNQ5U4COEFN


Let's explore the first neighborhood in your dataframe

In [58]:
etobicoke_data.loc[0, 'Neighborhood']

'Islington Avenue'

Get the neighborhood's latitude and longitude values

In [59]:
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Islington Avenue are 43.6678556, -79.5322424.


Let's create the GET request URL

In [63]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [64]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '610bfbbe2bc6b6754d47b008'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Edenbridge - Humber Valley',
  'headerFullLocation': 'Edenbridge - Humber Valley, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 43.676855609000015,
    'lng': -79.51982358836783},
   'sw': {'lat': 43.65885559099999, 'lng': -79.54466121163217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bfd53764cf820a13849ecf4',
       'name': "Java Joe's Village Cafe",
       'location': {'address': '1500 Islington Ave',
        'crossStreet': 'at Rathburn Rd',
        'lat': 43.662460906352436,
        'lng': -7

In [65]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-66-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Java Joe's Village Cafe,Café,43.662461,-79.532054
1,St Georges Golf and Country Club,Golf Course,43.674395,-79.537142
2,TD Canada Trust,Bank,43.662545,-79.531749
3,Shoppers Drug Mart,Pharmacy,43.663067,-79.531753
4,COBS Bread,Bakery,43.664940,-79.520485


In [67]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

12 venues were returned by Foursquare.


Explore neighborhoods in Etobicoke

Create a function to do the above procedure for all the neighborhoods in Etobicoke

In [68]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [69]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

Islington Avenue
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Westmount
Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips
Humber Bay Shores, Mimico South, New Toronto
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
Alderwood, Long Branch
Northwest
The Kingsway, Montgomery Road, Old Mill North
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea


In [73]:
print(etobicoke_venues.shape)
etobicoke_venues.head(15)

(55, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,LCBO,43.642099,-79.576592,Liquor Store
1,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,Starbucks,43.641312,-79.576924,Coffee Shop
2,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,The Beer Store,43.641313,-79.576925,Beer Store
3,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,Cafe Sympatico,43.641820,-79.576721,Café
4,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,Shoppers Drug Mart,43.641312,-79.576924,Pharmacy
5,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,Pizza Hut,43.641514,-79.576326,Pizza Place
6,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,Pet Valu,43.641667,-79.577050,Pet Store
7,Westmount,43.696319,-79.532242,Starbucks,43.696338,-79.533398,Coffee Shop
8,Westmount,43.696319,-79.532242,Mayflower Chinese Food,43.692753,-79.531566,Chinese Restaurant
9,Westmount,43.696319,-79.532242,Pizza Hut Etobicoke,43.696562,-79.533506,Pizza Place


In [72]:
etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",8,8,8,8,8,8
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",7,7,7,7,7,7
"Humber Bay Shores, Mimico South, New Toronto",11,11,11,11,11,11
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",1,1,1,1,1,1
"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",4,4,4,4,4,4
Northwest,5,5,5,5,5,5
"The Kingsway, Montgomery Road, Old Mill North",1,1,1,1,1,1
Westmount,8,8,8,8,8,8


#### Let's find out how many unique categories can be curated from all the returned venues

In [75]:
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 31 uniques categories.


Analyze each neighborhood

In [76]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighborhood,Bakery,Bar,Baseball Field,Beer Store,Bus Line,Café,Chinese Restaurant,Coffee Shop,Discount Store,Drugstore,Fast Food Restaurant,Fried Chicken Joint,Garden Center,Grocery Store,Gym,Intersection,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Truck Stop
0,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Examine the shape of the new dataframe size

In [78]:
etobicoke_onehot.shape

(55, 32)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [79]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,Bakery,Bar,Baseball Field,Beer Store,Bus Line,Café,Chinese Restaurant,Coffee Shop,Discount Store,Drugstore,Fast Food Restaurant,Fried Chicken Joint,Garden Center,Grocery Store,Gym,Intersection,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Truck Stop
0,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",0.000000,0.0,0.0,0.100000,0.00,0.000000,0.000,0.100000,0.000,0.0,0.100000,0.1,0.0,0.2,0.000000,0.000,0.100000,0.000000,0.000,0.00,0.00,0.000000,0.100000,0.100000,0.000,0.000,0.0,0.000000,0.0,0.100,0.0
1,"Alderwood, Long Branch",0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000,0.125000,0.000,0.0,0.000000,0.0,0.0,0.0,0.125000,0.000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.125000,0.250000,0.125,0.125,0.0,0.000000,0.0,0.125,0.0
2,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",0.000000,0.0,0.0,0.142857,0.00,0.142857,0.000,0.142857,0.000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000,0.142857,0.000000,0.000,0.00,0.00,0.142857,0.142857,0.142857,0.000,0.000,0.0,0.000000,0.0,0.000,0.0
3,"Humber Bay Shores, Mimico South, New Toronto",0.090909,0.0,0.0,0.000000,0.00,0.181818,0.000,0.000000,0.000,0.0,0.090909,0.0,0.0,0.0,0.090909,0.000,0.090909,0.090909,0.000,0.00,0.00,0.090909,0.090909,0.090909,0.000,0.000,0.0,0.090909,0.0,0.000,0.0
4,"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",0.000000,0.0,1.0,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.000,0.000,0.0,0.000000,0.0,0.000,0.0
5,"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",0.000000,0.0,0.0,0.000000,0.25,0.000000,0.000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.25,0.25,0.000000,0.000000,0.000000,0.000,0.000,0.0,0.000000,0.0,0.250,0.0
6,Northwest,0.000000,0.2,0.0,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.2,0.000000,0.0,0.2,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.000,0.000,0.2,0.000000,0.0,0.000,0.2
7,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.000000,0.000000,0.000,0.000,0.0,0.000000,1.0,0.000,0.0
8,Westmount,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.125,0.125000,0.125,0.0,0.000000,0.0,0.0,0.0,0.000000,0.125,0.000000,0.000000,0.125,0.00,0.00,0.000000,0.000000,0.125000,0.125,0.000,0.0,0.000000,0.0,0.125,0.0


Confirm the new size

In [81]:
etobicoke_grouped.shape

(9, 32)

Print each neighborhood along with the top 5 most common venues

In [82]:
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store   0.2
1  Fried Chicken Joint   0.1
2       Sandwich Place   0.1
3           Beer Store   0.1
4         Liquor Store   0.1


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1  Sandwich Place  0.12
2             Pub  0.12
3     Coffee Shop  0.12
4      Playground  0.12


----Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe----
          venue  freq
0  Liquor Store  0.14
1    Beer Store  0.14
2          Café  0.14
3     Pet Store  0.14
4   Coffee Shop  0.14


----Humber Bay Shores, Mimico South, New Toronto----
                  venue  freq
0                  Café  0.18
1                Bakery  0.09
2  Fast Food Restaurant  0.09
3          Liquor Store  0.09
4                   Gym  0.09


----Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The

Write a function to sort the venues in descending order

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [85]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",Grocery Store,Fried Chicken Joint,Sandwich Place,Beer Store,Liquor Store,Pharmacy,Coffee Shop,Pizza Place,Fast Food Restaurant,Bakery
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Pub,Coffee Shop,Playground,Pharmacy,Gym,Bakery,Mexican Restaurant,River
2,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",Liquor Store,Beer Store,Café,Pet Store,Coffee Shop,Pharmacy,Pizza Place,Playground,Mobile Phone Shop,Park
3,"Humber Bay Shores, Mimico South, New Toronto",Café,Bakery,Fast Food Restaurant,Liquor Store,Gym,Pet Store,Pharmacy,Pizza Place,Restaurant,Mexican Restaurant
4,"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",Baseball Field,Bakery,Liquor Store,Sandwich Place,River,Restaurant,Rental Car Location,Pub,Playground,Pizza Place


Cluster neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [86]:
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 3, 0, 4, 1, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [138]:
etobicoke_merged = etobicoke_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etobicoke_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",43.650943,-79.554724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",43.643515,-79.577201,2.0,Liquor Store,Beer Store,Café,Pet Store,Coffee Shop,Pharmacy,Pizza Place,Playground,Mobile Phone Shop,Park
3,M9P,Etobicoke,Westmount,43.696319,-79.532242,2.0,Intersection,Chinese Restaurant,Pizza Place,Playground,Coffee Shop,Discount Store,Sandwich Place,Middle Eastern Restaurant,Rental Car Location,Restaurant
4,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",43.688905,-79.554724,0.0,Sandwich Place,Bus Line,Park,Mobile Phone Shop,Bakery,Liquor Store,River,Restaurant,Rental Car Location,Pub


Visualize the resulting clusters

In [139]:
etobicoke_merged.replace(np.nan,0,inplace=True)

In [140]:
etobicoke_merged.rename(columns={'Cluster Labels': 'ClusterLabels'}, inplace=True)

In [142]:
#etobicoke_merged.ClusterLabels.astype(int)
etobicoke_merged['ClusterLabels']=etobicoke_merged['ClusterLabels'].fillna(0).astype(int)

In [145]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neighborhood'], etobicoke_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

Cluster 1

In [146]:
etobicoke_merged.loc[etobicoke_merged['ClusterLabels'] == 0, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,0,0,0,0,0,0,0,0,0,0,0
1,Etobicoke,0,0,0,0,0,0,0,0,0,0,0
4,Etobicoke,0,Sandwich Place,Bus Line,Park,Mobile Phone Shop,Bakery,Liquor Store,River,Restaurant,Rental Car Location,Pub


Cluster 2

In [147]:
etobicoke_merged.loc[etobicoke_merged['ClusterLabels'] == 1, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Etobicoke,1,River,Bakery,Liquor Store,Sandwich Place,Restaurant,Rental Car Location,Pub,Playground,Pizza Place,Pharmacy


Cluster 3

In [148]:
etobicoke_merged.loc[etobicoke_merged['ClusterLabels'] == 2, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Etobicoke,2,Liquor Store,Beer Store,Café,Pet Store,Coffee Shop,Pharmacy,Pizza Place,Playground,Mobile Phone Shop,Park
3,Etobicoke,2,Intersection,Chinese Restaurant,Pizza Place,Playground,Coffee Shop,Discount Store,Sandwich Place,Middle Eastern Restaurant,Rental Car Location,Restaurant
5,Etobicoke,2,Café,Bakery,Fast Food Restaurant,Liquor Store,Gym,Pet Store,Pharmacy,Pizza Place,Restaurant,Mexican Restaurant
6,Etobicoke,2,Grocery Store,Fried Chicken Joint,Sandwich Place,Beer Store,Liquor Store,Pharmacy,Coffee Shop,Pizza Place,Fast Food Restaurant,Bakery
7,Etobicoke,2,Pizza Place,Sandwich Place,Pub,Coffee Shop,Playground,Pharmacy,Gym,Bakery,Mexican Restaurant,River


Cluster 4

In [149]:
etobicoke_merged.loc[etobicoke_merged['ClusterLabels'] == 3, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Etobicoke,3,Baseball Field,Bakery,Liquor Store,Sandwich Place,River,Restaurant,Rental Car Location,Pub,Playground,Pizza Place


Cluster 5

In [150]:
etobicoke_merged.loc[etobicoke_merged['ClusterLabels'] == 4, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Etobicoke,4,Truck Stop,Drugstore,Bar,Rental Car Location,Garden Center,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place
